In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from lightgbm import LGBMRegressor

import gc
import time
import random
import eli5
from eli5.sklearn import PermutationImportance
import shap

warnings.filterwarnings("ignore")

In [16]:
import pandas as pd
from smart_open import smart_open
import os
from configparser import ConfigParser

config = ConfigParser()

config_file = os.path.join(os.path.dirname('__file__'), 'config.ini')

config.read(config_file)
default = config['aws.data']
aws_key = default['accessKey']
aws_secret = default['secretAccessKey']

bucket_name = 'pubg-dataset-files'
object_key = 'final_train.csv'

path = 's3://{}:{}@{}/{}'.format(aws_key, aws_secret, bucket_name, object_key)

df = pd.read_csv(smart_open(path))

In [17]:
df.head()

,Unnamed: 0,totalDistance,weaponsAcquired,healsAndBoosts,longestKill,killsNorm,assists,DBNOs,headshotKills,revives,vehicleDestroys,winPlacePerc,killPlace,numGroups
0,0,244.8000,1,0.0,0.00,0.00,0,0,0,0,0,0.4444,60,26
1,1,1445.0445,5,0.0,0.00,0.00,0,0,0,0,0,0.6400,57,25
2,2,161.8000,2,0.0,0.00,0.00,1,0,0,0,0,0.7755,47,47
3,3,202.7000,3,0.0,0.00,0.00,0,0,0,0,0,0.1667,75,30
4,4,49.7500,2,0.0,58.53,1.03,0,0,0,0,0,0.1875,45,95


In [18]:
df=df.drop(['Unnamed: 0'], axis=1)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000662 entries, 0 to 4000661
Data columns (total 13 columns):
totalDistance      float64
weaponsAcquired    int64
healsAndBoosts     float64
longestKill        float64
killsNorm          float64
assists            int64
DBNOs              int64
headshotKills      int64
revives            int64
vehicleDestroys    int64
winPlacePerc       float64
killPlace          int64
numGroups          int64
dtypes: float64(5), int64(8)
memory usage: 396.8 MB


In [20]:
df.head()

,totalDistance,weaponsAcquired,healsAndBoosts,longestKill,killsNorm,assists,DBNOs,headshotKills,revives,vehicleDestroys,winPlacePerc,killPlace,numGroups
0,244.8000,1,0.0,0.00,0.00,0,0,0,0,0,0.4444,60,26
1,1445.0445,5,0.0,0.00,0.00,0,0,0,0,0,0.6400,57,25
2,161.8000,2,0.0,0.00,0.00,1,0,0,0,0,0.7755,47,47
3,202.7000,3,0.0,0.00,0.00,0,0,0,0,0,0.1667,75,30
4,49.7500,2,0.0,58.53,1.03,0,0,0,0,0,0.1875,45,95


In [21]:
df.describe()

,totalDistance,weaponsAcquired,healsAndBoosts,longestKill,killsNorm,assists,DBNOs,headshotKills,revives,vehicleDestroys,winPlacePerc,killPlace,numGroups
count,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06,4.000662e+06
mean,1.764573e+03,3.656119e+00,2.475420e+00,2.297664e+01,9.811855e-01,2.332049e-01,6.576512e-01,2.262071e-01,1.647025e-01,7.913440e-03,4.727735e-01,4.761134e+01,4.302187e+01
std,2.177866e+03,2.419845e+00,3.868107e+00,5.089997e+01,1.706735e+00,5.856123e-01,1.143259e+00,5.954039e-01,4.720016e-01,9.256458e-02,3.073818e-01,2.746340e+01,2.329612e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,1.585000e+02,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-01,2.400000e+01,2.700000e+01
50%,7.896000e+02,3.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.583000e-01,4.700000e+01,3.000000e+01
75%,2.729000e+03,5.000000e+00,4.000000e+00,2.129000e+01,1.090000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.407000e-01,7.100000e+01,4.700000e+01
max,2.252100e+04,7.800000e+01,4.900000e+01,9.999000e+02,5.820000e+01,1.700000e+01,3.300000e+01,2.300000e+01,3.200000e+01,5.000000e+00,1.000000e+00,1.010000e+02,1.000000e+02
